In [1]:
import json
import pandas as pd
import streamlit as st
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [3]:
def read_expenses():
    # Set up the service account credentials
    service_account_info = json.loads(st.secrets.service_account_info)

    creds = service_account.Credentials.from_service_account_info(
        service_account_info, scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
    )

    # Create a Sheets API client
    sheets_service = build("sheets", "v4", credentials=creds)

    # Specify the file ID of the Google Sheets file you want to read
    file_id = "1WOcaZOyHYGhebc80gsgoLjw4ofabIymCJkq7smgvsH8"

    # Specify the range of cells you want to read (e.g., "Sheet1!A1:C10")
    range_name = "gastos"

    # Make a request to get the values from the specified range
    result = sheets_service.spreadsheets().values().get(spreadsheetId=file_id, range=range_name).execute()

    # Get the values from the result
    data = result.get("values", [])

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


df = read_expenses()

In [14]:
import plotly.express as px

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Convert fecha to datetime
df["fecha"] = pd.to_datetime(df["fecha"])
df["monto"] = pd.to_numeric(df["monto"])
# Add month and week columns
df["month"] = df["fecha"].dt.to_period("M").astype(str)
df["week"] = df["fecha"].dt.to_period("W").astype(str)

# Monthly expenses bar chart
monthly_expenses = df.groupby("month")["monto"].sum().reset_index()
fig_monthly_expenses = px.bar(monthly_expenses, x="month", y="monto", title="Monthly Expenses")
fig_monthly_expenses

In [31]:
2 * 1600

3200

In [17]:
# Bar chart of top consumed products
top_consumed_products = (
    df.groupby("descripcion")["cantidad"].sum().reset_index().sort_values("cantidad", ascending=False)
)
fig_top_consumed_products = px.bar(
    top_consumed_products.head(10), x="descripcion", y="cantidad", title="Top Consumed Products"
)
fig_top_consumed_products.show()

In [10]:
df

,fecha,descripcion,cantidad,unidad,monto,categoria,comercio,month,week
0,2024-04-20,LECHE LONCOLECHE,4,unit,4200,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
1,2024-04-20,YOGHURT SOPROLE GOLD,4,unit,2500,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
2,2024-04-20,COMPOTA VIVO MANGO,12,unit,7824,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
3,2024-04-20,COIMPOTA VIVO FRAMBUESA,8,unit,2608,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
4,2024-04-20,COCA COLA ORIGINAL LATA,6,unit,4073,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
5,2024-04-20,PRESTIGIO NESTLE,5,unit,2199,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
6,2024-04-20,Pan doblada granel,2,unit,701,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
7,2024-04-20,Pan hallulla granel,2,unit,396,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
8,2024-04-20,ENSALADA REPOLLO,2,unit,1590,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21
9,2024-04-20,AJOS,1,unit,1290,FOOD,JUMBO,2024-04,2024-04-15/2024-04-21


In [9]:
df.dtypes

fecha          datetime64[ns]
descripcion            object
cantidad               object
unidad                 object
monto                  object
categoria              object
comercio               object
month               period[M]
week            period[W-SUN]
dtype: object

In [8]:
fig_monthly_expenses.show()

TypeError: Object of type Period is not JSON serializable

In [15]:
# Monthly expenses by comercio bar chart
monthly_expenses_comercio = df.groupby(["month", "comercio"])["monto"].sum().reset_index()
fig_monthly_expenses_comercio = px.bar(
    monthly_expenses_comercio, x="month", y="monto", color="comercio", title="Monthly Expenses by Comercio"
)
fig_monthly_expenses_comercio.show()

# Time series plot of expenses by month
monthly_expenses_ts = df.groupby(["month", "categoria"])["monto"].sum().reset_index()
fig_monthly_expenses_ts = px.line(
    monthly_expenses_ts, x="month", y="monto", color="categoria", title="Monthly Expenses Time Series"
)
fig_monthly_expenses_ts.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label="All Categories",
                    method="update",
                    args=[{"visible": [True] * len(fig_monthly_expenses_ts.data)}],
                ),
                dict(
                    label="Food",
                    method="update",
                    args=[
                        {"visible": [True if trace.name == "food" else False for trace in fig_monthly_expenses_ts.data]}
                    ],
                ),
            ],
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
        )
    ]
)
fig_monthly_expenses_ts.show()

# Time series plot of expenses by week
weekly_expenses_ts = df.groupby(["week", "categoria"])["monto"].sum().reset_index()
fig_weekly_expenses_ts = px.line(
    weekly_expenses_ts, x="week", y="monto", color="categoria", title="Weekly Expenses Time Series"
)
fig_weekly_expenses_ts.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label="All Categories",
                    method="update",
                    args=[{"visible": [True] * len(fig_weekly_expenses_ts.data)}],
                ),
                dict(
                    label="Food",
                    method="update",
                    args=[
                        {"visible": [True if trace.name == "food" else False for trace in fig_weekly_expenses_ts.data]}
                    ],
                ),
            ],
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
        )
    ]
)
fig_weekly_expenses_ts.show()

# Pie chart of expenses by category per month
monthly_expenses_category = df.groupby(["month", "categoria"])["monto"].sum().reset_index()
fig_monthly_expenses_category = px.pie(
    monthly_expenses_category,
    names="categoria",
    values="monto",
    title="Monthly Expenses by Category",
    facet_col="month",
    facet_col_wrap=3,
)
fig_monthly_expenses_category.show()

# Pie chart of expenses by category per week
weekly_expenses_category = df.groupby(["week", "categoria"])["monto"].sum().reset_index()
fig_weekly_expenses_category = px.pie(
    weekly_expenses_category,
    names="categoria",
    values="monto",
    title="Weekly Expenses by Category",
    facet_col="week",
    facet_col_wrap=3,
)
fig_weekly_expenses_category.show()

# Most expensive products in the Food category per rolling month
df_food = df[df["categoria"] == "food"]
rolling_month_expenses = (
    df_food.groupby(["descripcion", pd.Grouper(key="fecha", freq="M")])["monto"].sum().reset_index()
)
top_products_per_month = rolling_month_expenses.loc[rolling_month_expenses.groupby("fecha")["monto"].idxmax()]
fig_top_products_per_month = px.bar(
    top_products_per_month,
    x="fecha",
    y="monto",
    color="descripcion",
    title="Most Expensive Food Products per Rolling Month",
)
fig_top_products_per_month.show()

In [8]:
st.dataframe(df)

2024-04-22 18:24:39.564 
  command:

    streamlit run /home/cecilvega/DataspellProjects/venvs/venv311/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [7]:
df

,FECHA,DESCRIPCION,CANTIDAD,UNIDAD,MONTO,CATEGORIA,COMERCIO
0,2024-04-20,LECHE LONCOLECHE,4,unit,4200,FOOD,JUMBO
1,2024-04-20,YOGHURT SOPROLE GOLD,4,unit,2500,FOOD,JUMBO
2,2024-04-20,COMPOTA VIVO MANGO,12,unit,7824,FOOD,JUMBO
3,2024-04-20,COIMPOTA VIVO FRAMBUESA,8,unit,2608,FOOD,JUMBO
4,2024-04-20,COCA COLA ORIGINAL LATA,6,unit,4073,FOOD,JUMBO
5,2024-04-20,PRESTIGIO NESTLE,5,unit,2199,FOOD,JUMBO
6,2024-04-20,Pan doblada granel,2,unit,701,FOOD,JUMBO
7,2024-04-20,Pan hallulla granel,2,unit,396,FOOD,JUMBO
8,2024-04-20,ENSALADA REPOLLO,2,unit,1590,FOOD,JUMBO
9,2024-04-20,AJOS,1,unit,1290,FOOD,JUMBO


In [ ]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Set up the OAuth 2.0 flow


# Run the OAuth 2.0 flow to obtain credentials
creds = flow.run_local_server(port=0)

# Create a Drive API client
drive_service = build("drive", "v3", credentials=creds)

# Specify the file ID of the file you want to read
file_id = "your_file_id_here"

# Make a request to get the file metadata
file_metadata = drive_service.files().get(fileId=file_id).execute()

# Get the file content
file_content = drive_service.files().get_media(fileId=file_id).execute()

# Process the file content as needed
print(file_content.decode("utf-8"))

In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Set up the credentials (you'll need to create and configure OAuth 2.0 credentials)

# Create a Drive API client
drive_service = build("drive", "v3", credentials=creds)
# Specify the file ID of the file you want to read
file_id = "1WOcaZOyHYGhebc80gsgoLjw4ofabIymCJkq7smgvsH8"

# Make a request to get the file metadata
file_metadata = drive_service.files().get(fileId=file_id).execute()

# Get the file content
file_content = drive_service.files().get_media(fileId=file_id).execute()

# Process the file content as needed
print(file_content.decode("utf-8"))

In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Set up the credentials (you'll need to create and configure OAuth 2.0 credentials)

In [ ]:
creds

In [ ]:
# Create a Drive API client
drive_service = build("drive", "v3", credentials=creds)

# Specify the file ID of the file you want to read
file_id = "your_file_id_here"

# Make a request to get the file metadata
file_metadata = drive_service.files().get(fileId=file_id).execute()

# Get the file content
file_content = drive_service.files().get_media(fileId=file_id).execute()

# Process the file content as needed
print(file_content.decode("utf-8"))